In [1]:
def representation(x):
    try:
        if x.lower()=="x":
            return " X "
        if x.lower()=="o":
            return " O "
    except:
        pass
    return "   "

In [2]:
def print_state(state):
    sep = "_"*11+"\n"
    ret = []
    n=0
    for row in state:
        ret.append("|".join(map(representation,row)))
        n+=1
        if n<3:
                ret.append(sep)
    for line in ret:
        print(line)

In [3]:
state = (('x','o',0.0),('o','x',0.0),(0.0,'o','x'))
print_state(state)

 X | O |   
___________

 O | X |   
___________

   | O | X 


In [4]:
def has_won(state):
    players = ['x', 'o']
    for i in [0,1]:
        for row in state:
            if row==tuple(players[i]*3): # _
                return i, True
        for cols in [0, 1, 2]:
            if state[0][cols]==state[1][cols] and state[2][cols]==state[0][cols] and state[0][cols]==players[i]: # |
                return i, True
        if state[0][0]==state[1][1] and state[0][0]==state[2][2] and state[0][0]==players[i]:   # \
                return i, True
        if state[2][0]==state[1][1] and state[2][0]==state[0][2] and state[0][2]==players[i]:   # /
                return i, True
            
    return -1, False

In [5]:
has_won(state)

(0, True)

In [6]:
state = (('o','o','o'),('o','x',0.0),(0.33,'o','x'))
print_state(state)
has_won(state)

 O | O | O 
___________

 O | X |   
___________

   | O | X 


(1, True)

In [7]:
state = (('o','x','o'),('o','x',0.0),('o','o','x'))
print_state(state)
has_won(state)

 O | X | O 
___________

 O | X |   
___________

 O | O | X 


(1, True)

In [8]:
state = (('o','x','x'),('o','x',0.0),('x','o','x'))
print_state(state)
has_won(state)

 O | X | X 
___________

 O | X |   
___________

 X | O | X 


(0, True)

In [9]:
state = (('o','x','o'),('o','x',0.0),('x','o','x'))
print_state(state)
has_won(state)

 O | X | O 
___________

 O | X |   
___________

 X | O | X 


(-1, False)

In [10]:
def is_full(state):
    for row in state:
        for v in row:
            if v!="x" and v!="o":
                return False
    return True

In [11]:
is_full((('o','x','o'),('o','x',0.0),('o','o','x')))

False

In [12]:
is_full((('o','x','o'),('o','x','x'),('o','o','x')))

True

In [13]:
class Player:
    def move(state):
        return state

In [14]:
def is_valid_move(state, x, y):
    row = state[x]
    if row[y]!="x" and row[y]!="o":
        return True
    return False

In [15]:
def new_state(state, x, y, character):
    l = []
    for row in state:
        l.append(list(row))
    l[x][y] = character
    return (tuple(l[0]), tuple(l[1]), tuple(l[2]))

In [16]:
state = (('o','x','o'),('o','x',0.0),('o','o','x'))
new_state(state, 1, 2, "x")

(('o', 'x', 'o'), ('o', 'x', 'x'), ('o', 'o', 'x'))

In [24]:
class HumanPlayer(Player):
    def __init__(self, character):
        self.character=character
        
    def move(self, state):
        print_state(state)
        flag = False
        while not flag:
            print()
            x = int(input(f"({self.character}) row: "))
            y = int(input(f"({self.character}) col: "))
            flag=is_valid_move(state, x, y)
        print()
        return new_state(state, x, y, self.character)

    def learn(self, won):
        pass

In [ ]:
h = HumanPlayer("x")
print_state(
    h.move((('o','x','o'),
            ('o','x',0.0),
            ('o','o','x')))
)

In [53]:
def do_game(player0, player1, stats, verbose=False):
    state = ((0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0))
    ended = False
    full = False
    won = -1
    while not ended and not full:
        state = player0.move(state)
        who, ended = has_won(state)
        full = is_full(state)        
        if not ended and not full:
            state = player1.move(state)
            who, ended = has_won(state)
            full = is_full(state)
    if ended:
        stats.append("WIN 0" if who==0 else "WIN 1")
        player0.learn(1.0 if who==0 else 0.0)
        player1.learn(1.0 if who==1 else 0.0)
    if full and not ended:
        stats.append("DRAW")    
        player0.learn(0.5)
        player1.learn(0.5)
    if verbose:
        print("DRAW" if full else f"{who} has won!")

In [ ]:
stats = []
do_game(HumanPlayer('x'), HumanPlayer('o'), stats, True)

In [18]:
import random

print([random.randrange(3) for x in range(100)])

[0, 0, 0, 1, 0, 2, 0, 1, 1, 1, 0, 0, 2, 1, 0, 2, 1, 2, 0, 0, 2, 2, 1, 0, 0, 1, 1, 0, 2, 1, 0, 1, 0, 2, 2, 1, 0, 1, 0, 0, 2, 0, 2, 2, 1, 2, 0, 0, 0, 2, 1, 2, 0, 0, 2, 2, 1, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 2, 2, 1, 1, 2, 2, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 2, 1, 1, 0, 2, 0, 0, 0, 1, 2, 1, 1, 1]


In [19]:
class RandomPlayer(Player):
    def __init__(self, character):
        self.character=character
        
    def move(self, state):
        flag = False
        while not flag:
            x = random.randrange(3)
            y = random.randrange(3)
            flag=is_valid_move(state, x, y)
       
        return new_state(state, x, y, self.character)
    
    def learn(self, won):
        pass

In [39]:
do_game(HumanPlayer('x'), RandomPlayer('o'), stats, True)

   |   |   
___________

   |   |   
___________

   |   |   



(x) row:  1
(x) col:  1



   |   |   
___________

   | X | O 
___________

   |   |   



(x) row:  0
(x) col:  0



 X |   | O 
___________

   | X | O 
___________

   |   |   



(x) row:  2
(x) col:  2



0 has won!


In [20]:
from collections import Counter

stats = []

for i in range(10_000):
    do_game(RandomPlayer('x'), RandomPlayer('o'), stats, False)
    
print(Counter(stats))

Counter({'WIN 0': 5847, 'WIN 1': 2860, 'DRAW': 1293})


In [81]:
from random import sample

α = 0.1
β = 0.5

class SimplePlayer(Player):
    def __init__(self, character):
        self.character=character
        self.q_table={}
        self.previous_state=((0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0))
        self.current_state=((0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0))
        
    def initialize_q_table(self, state):    
        actions = {}   
        for x in range(3):
            for y in range(3):
                if is_valid_move(state, x, y):
                    actions[new_state(state, x, y, self.character)] = β
        self.q_table[state] = actions
        print(actions)
        return actions
        
    def move(self, state):
        flag = False
        
        actions = self.q_table.get(state)
        if not actions:
            actions = self.initialize_q_table(state)

        best_q = max(actions.values())
        
        best_actions = [ action for action, weight in actions.items() if weight==best_q]
 
        self.previous_state = state
        self.current_state = sample(best_actions, 1)[0]

        return self.current_state
    
    def learn(self, learned_weight):
        self.q_table[self.previous_state][self.current_state] = learned_weight
        
        for state, actions in self.q_table.items():
            for next_move, weight in actions.items():
                print(next_move)
                
                next_move_actions = self.q_table.get(next_move)
                if next_move_actions:
                    best_next_q = max(next_move_actions.values())
                    print(best_next_q)
                    actions[next_move] = (1-α)*weight + α*best_next_q 
                    self.q_table[state] = actions
       

In [82]:
stats = []
player = SimplePlayer('x')


In [83]:
for i in range(5):
    do_game(player, RandomPlayer('o'), stats, False)
    
print(Counter(stats))
vals = [ max(v.values()) for v in player.q_table.values()]
print(vals)

{(('x', 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 'x', 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 0.0, 'x'), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 0.0, 0.0), ('x', 0.0, 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 0.0, 0.0), (0.0, 'x', 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 0.0, 0.0), (0.0, 0.0, 'x'), (0.0, 0.0, 0.0)): 0.5, ((0.0, 0.0, 0.0), (0.0, 0.0, 0.0), ('x', 0.0, 0.0)): 0.5, ((0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 'x', 0.0)): 0.5, ((0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 'x')): 0.5}
{(('x', 'x', 0.0), ('o', 0.0, 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 'x', 'x'), ('o', 0.0, 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 'x', 0.0), ('o', 'x', 0.0), (0.0, 0.0, 0.0)): 0.5, ((0.0, 'x', 0.0), ('o', 0.0, 'x'), (0.0, 0.0, 0.0)): 0.5, ((0.0, 'x', 0.0), ('o', 0.0, 0.0), ('x', 0.0, 0.0)): 0.5, ((0.0, 'x', 0.0), ('o', 0.0, 0.0), (0.0, 'x', 0.0)): 0.5, ((0.0, 'x', 0.0), ('o', 0.0, 0.0), (0.0, 0.0, 'x')): 0.5}
{(('x', 'x', 0.0), ('o', 0.0, 0.0), (0.0, 'x', 'o')): 0.5, ((0.0, 'x',

In [69]:
stats = []
player = SimplePlayer('o')
for i in range(1_000):
    do_game(RandomPlayer('x'), player,  stats, False)
    
print(Counter(stats))


Counter({'WIN 0': 509, 'WIN 1': 364, 'DRAW': 127})
